In [1]:
###########################################
# IMPORTS
###########################################
from simtk.openmm import app
from simtk import unit, openmm
import numpy as np
import os
from nose.tools import nottest

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

from perses.annihilation.relative import HybridTopologyFactory
from perses.rjmc.geometry import FFAllAngleGeometryEngine
from perses.rjmc.topology_proposal import SmallMoleculeSetProposalEngine, TopologyProposal
from perses.tests import utils
import openeye.oechem as oechem
from openmmtools import alchemy
from openmmtools.states import ThermodynamicState, SamplerState, CompoundThermodynamicState
import openmmtools.mcmc as mcmc
import openmmtools.cache as cache
from unittest import skipIf

import pymbar.timeseries as timeseries

import copy
import pymbar

running_on_github_actions = os.environ.get('GITHUB_ACTIONS', None) == 'true'

try:
    cache.global_context_cache.platform = openmm.Platform.getPlatformByName("Reference")
except Exception:
    cache.global_context_cache.platform = openmm.Platform.getPlatformByName("Reference")

#############################################
# CONSTANTS
#############################################
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
CARBON_MASS = 12.01
ENERGY_THRESHOLD = 1e-1
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CPU")

RDKit WARNING: [19:24:37] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
def compare_energies(mol_name="naphthalene", ref_mol_name="benzene",atom_expression=['Hybridization'],bond_expression=['Hybridization']):
    """
    Make an atom map where the molecule at either lambda endpoint is identical, and check that the energies are also the same.
    """
    from openmmtools.constants import kB
    from openmmtools import alchemy, states
    from perses.rjmc.topology_proposal import SmallMoleculeSetProposalEngine
    from perses.annihilation.relative import HybridTopologyFactory
    from perses.rjmc.geometry import FFAllAngleGeometryEngine
    import simtk.openmm as openmm
    from perses.utils.openeye import iupac_to_oemol, extractPositionsFromOEMol, generate_conformers
    from perses.utils.openeye import generate_expression
    from openmmforcefields.generators import SystemGenerator
    from openmoltools.forcefield_generators import generateTopologyFromOEMol
    from perses.tests.utils import validate_endstate_energies
    temperature = 300*unit.kelvin
    # Compute kT and inverse temperature.
    kT = kB * temperature
    beta = 1.0 / kT
    ENERGY_THRESHOLD = 1e-6

    atom_expr, bond_expr = generate_expression(atom_expression), generate_expression(bond_expression)

    mol = iupac_to_oemol(mol_name)
    mol = generate_conformers(mol, max_confs=1)

    refmol = iupac_to_oemol(ref_mol_name)
    refmol = generate_conformers(refmol,max_confs=1)

    from openforcefield.topology import Molecule
    molecules = [Molecule.from_openeye(oemol) for oemol in [refmol, mol]]
    barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, temperature, 50)
    forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
    forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 1e-4, 'nonbondedMethod': app.NoCutoff, 'constraints' : app.HBonds, 'hydrogenMass' : 4 * unit.amus}

    system_generator = SystemGenerator(forcefields = forcefield_files, barostat=barostat, forcefield_kwargs=forcefield_kwargs,
                                         small_molecule_forcefield = 'gaff-2.11', molecules=molecules, cache=None)


    topology = generateTopologyFromOEMol(refmol)
    system = system_generator.create_system(topology)
    positions = extractPositionsFromOEMol(refmol)
    
    modeller = app.Modeller(topology, positions)
    modeller.addSolvent(system_generator.forcefield, model='tip3p', padding=9.0*unit.angstroms)
    topology = modeller.getTopology()
    positions = modeller.getPositions()
    positions = unit.quantity.Quantity(value = np.array([list(atom_pos) for atom_pos in positions.value_in_unit_system(unit.md_unit_system)]), unit = unit.nanometers)
    system = system_generator.create_system(topology)


    proposal_engine = SmallMoleculeSetProposalEngine([refmol, mol], system_generator)
    proposal = proposal_engine.propose(system, topology, atom_expr = atom_expr, bond_expr = bond_expr)
    geometry_engine = FFAllAngleGeometryEngine()
    new_positions, _ = geometry_engine.propose(proposal, positions, beta = beta, validate_energy_bookkeeping = False)
    _ = geometry_engine.logp_reverse(proposal, new_positions, positions, beta)
    #make a topology proposal with the appropriate data:

    factory = HybridTopologyFactory(proposal, positions, new_positions)
    if not proposal.unique_new_atoms:
        assert geometry_engine.forward_final_context_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.forward_final_context_reduced_potential})"
        assert geometry_engine.forward_atoms_with_positions_reduced_potential == None, f"There are no unique new atoms but the geometry_engine's forward atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.forward_atoms_with_positions_reduced_potential})"
        vacuum_added_valence_energy = 0.0
    else:
        added_valence_energy = geometry_engine.forward_final_context_reduced_potential - geometry_engine.forward_atoms_with_positions_reduced_potential

    if not proposal.unique_old_atoms:
        assert geometry_engine.reverse_final_context_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's final context reduced potential is not None (i.e. {self._geometry_engine.reverse_final_context_reduced_potential})"
        assert geometry_engine.reverse_atoms_with_positions_reduced_potential == None, f"There are no unique old atoms but the geometry_engine's atoms-with-positions-reduced-potential in not None (i.e. { self._geometry_engine.reverse_atoms_with_positions_reduced_potential})"
        subtracted_valence_energy = 0.0
    else:
        subtracted_valence_energy = geometry_engine.reverse_final_context_reduced_potential - geometry_engine.reverse_atoms_with_positions_reduced_potential

    zero_state_error, one_state_error = validate_endstate_energies(factory._topology_proposal, factory, added_valence_energy, subtracted_valence_energy, beta = 1.0/(kB*temperature), ENERGY_THRESHOLD = ENERGY_THRESHOLD, platform = openmm.Platform.getPlatformByName('Reference'))
    return factory

In [3]:
import pickle
for derivative in ['fluorobenzene', 'methylbenzene', 'phenol', 'aniline', 'ethylbenzene', 'anisole', '1-methylaniline']:
    factory = compare_energies(mol_name=derivative, ref_mol_name="benzene")
    with open(f"benzene_{derivative}.solvent.factory.pkl", 'wb') as f:
        pickle.dump(factory, f)

DEBUG:perses.forcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:openmmforcefields.generators.template_generators:Requested to generate parameters for residue <Residue 0 (MOL) of chain 0>
INFO:openmmforcefields.generators.template_generators:Generating a residue template for [H]c1c(c(c(c(c1[H])[H])[H])[H])[H]
DEBUG:openmmforcefields.generators.template_generators:Total charge is 0
DEBUG:openmmforcefields.generators.template_generators:Using user-provided charges because partial charges are nonzero...
DEBUG:openmmforcefields.generators.template_generators:Generating a conformer...
DEBUG:openmmforcefields.generators.template_generators:Running antechamber...
DEBUG:openmmforcefields.generators.template_generators:antechamber -i in.mdl -fi mdl -o out.mol2 -fo mol2 -s 0 -at 2 -dr no
DEBUG:openmmforcefields.generators.template_generators:
Welcome to antechamber 19.0: molecular input file processor.


DEBUG:openmmforcefields.generators.template_generators:parmchk

added energy components: [('CustomBondForce', 0.043200196540455024), ('CustomAngleForce', 0.19067520235990218), ('CustomTorsionForce', 0.16743187451119573), ('CustomBondForce', -2.7497402825035904)]


INFO:geometry:There are 1 new atoms
INFO:geometry:	reduced angle potential = 0.000284098811169791.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 6 bond forces in the no-nonbonded final system
INFO:geometry:	there are 18 angle forces in the no-nonbonded final system
INFO:geometry:	there are 30 torsion forces in the no-nonbonded final system
INFO:geometry:reverse final system defined with 0 neglected angles.
INFO:geometry:total reduced potential before atom placement: 0.041185464175714184
INFO:geometry:total reduced energy added from growth system: 2.271382326312849
INFO:geometry:final reduced energy 2.3125677904885626
INFO:geomet

added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.0007369126623679488), ('CustomTorsionForce', 3.760243033931317e-06), ('CustomBondForce', 2.270641653407447)]


INFO:relative:Handling unique_new/old interaction exceptions...
INFO:relative:There are old or new system exceptions...proceeding.
INFO:relative:	handle_old_new_exceptions: <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x7f6946a06180> > added to hybrid system


...

added energy components: [('CustomBondForce', 0.7695199870002831), ('CustomAngleForce', 3.8044787571427117), ('CustomTorsionForce', 4.9985344652614625), ('CustomBondForce', -47.191169577452094)]


INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 6 bond forces in the no-nonbonded final system
INFO:geometry:	there are 18 angle forces in the no-nonbonded final system
INFO:geometry:	there are 30 torsion forces in the no-nonbonded final system
INFO:geometry:reverse final system defined with 0 neglected angles.
INFO:geometry:total reduced potential before atom placement: 0.041185464175714184
INFO:geometry:total reduced energy added from growth system: 2.271382326312849
INFO:geometry:final reduced energy 2.3125677904885626
INFO:geometry:sum of energies: 2.3125677904885635
INFO:geometry:magnitude of difference in the energies: 8.881784197001252e-16
INFO:geometry:Final logp_proposal: 6.211746958702406
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'Ha

added energy components: [('CustomBondForce', 0.0), ('CustomAngleForce', 0.0007369126623679488), ('CustomTorsionForce', 3.760243033931317e-06), ('CustomBondForce', 2.270641653407447)]


INFO:relative:Handling unique_new/old interaction exceptions...
INFO:relative:There are old or new system exceptions...proceeding.
INFO:relative:	handle_old_new_exceptions: <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x7f6946a06f00> > added to hybrid system


			HarmonicBondForce: 0.03730173948947656
			HarmonicAngleForce: 0.004507905000805822
			PeriodicTorsionForce: 0.00011649259083367851
			NonbondedForce: 592.9307398970677
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
added forces:592.9726660341488
rp: 593.2668645687512
			CustomBondForce: 0.03730173948947656
			HarmonicBondForce: 0.7695199870002831
			CustomAngleForce: 0.003770992338437873
			HarmonicAngleForce: 3.805215669805079
			CustomTorsionForce: 0.0
			PeriodicTorsionForce: 4.998650957852295
			NonbondedForce: 557.3360817188267
			CustomNonbondedForce: -11.596554098117418
			CustomBondForce: 0.0
			AndersenThermostat: 0.0
			MonteCarloBarostat: 0.0
added forces:555.3539869671948
rp: 555.6482282007039
			CustomBondForce: 0.03730173948947656
			HarmonicBondForce: 0.7695199870002831
			CustomAngleForce: 0.003770992338437873
			HarmonicAngleForce: 3.805215669805079
			CustomTorsionForce: 0.0
			PeriodicTorsionForce: 4.998650957852295
			NonbondedForce: 569.9990228851668
			C